In [2]:
import pandas as pd

In [3]:
df_dpm = pd.read_csv('dpm-opt-experiments.csv')
df_dpm_heur = pd.read_csv('dpm-experiments-heur.csv')

df_dfm = pd.read_csv('dfm-opt-experiments.csv')
df_dfm_heur = pd.read_csv('dfm-experiments-heur.csv')

df_custom = pd.read_csv('custom-opt-experiments.csv')
df_custom_heur = pd.read_csv('custom-experiments-heur.csv')

In [4]:
def get_groups(x: str):
    y = x.split('-')[1]
    return ((int(y) - 1) // 10) + 1


def insert_group(df: pd.DataFrame) -> None:
    df.insert(0, 'group', df['name'].apply(get_groups))


insert_group(df_dpm)
insert_group(df_dpm_heur)
insert_group(df_dfm)
insert_group(df_dfm_heur)
insert_group(df_custom)
insert_group(df_custom_heur)

In [5]:
df_dpm_heur.columns

Index(['group', 'name', 'neh', 'neh_ls', 'multistart', 'iga'], dtype='object')

In [6]:
def merge_metrics(df_bnb: pd.DataFrame, df_heur: pd.DataFrame) -> pd.DataFrame:
    merged = pd.merge(
        df_bnb[['name', 'incumbent', 'lower_bound']],
        df_heur[['group', 'name', 'neh', 'neh_ls', 'multistart', 'iga']],
        on='name',
    )

    for col in ['neh', 'neh_ls', 'multistart', 'iga']:
        merged[f'rel_gap_{col}'] = (
            (merged[col] - merged['lower_bound']) / merged['lower_bound']
        ) * 100
        merged[f'rel_gap_{col}'] = merged[f'rel_gap_{col}'].apply(
            lambda x: round(x, 2)
        )
        merged[f'arpd_{col}'] = (
            (merged[col] - merged['incumbent']) / merged['incumbent']
        ) * 100
        merged[f'arpd_{col}'] = merged[f'arpd_{col}'].apply(
            lambda x: round(x, 2)
        )

    return merged


dpm_merged = merge_metrics(df_dpm, df_dpm_heur)
dfm_merged = merge_metrics(df_dfm, df_dfm_heur)
custom_merged = merge_metrics(df_custom, df_custom_heur)

In [7]:
def agg_dataframe(df_out: pd.DataFrame) -> pd.DataFrame:
    return df_out.groupby('group', as_index=False).agg({
        'rel_gap_neh': 'mean',
        'arpd_neh': 'mean',
        'rel_gap_neh_ls': 'mean',
        'arpd_neh_ls': 'mean',
        'rel_gap_multistart': 'mean',
        'arpd_multistart': 'mean',
        'rel_gap_iga': 'mean',
        'arpd_iga': 'mean',
    })

In [8]:
def export_arpd(df: pd.DataFrame, filename: str) -> None:
    agg_df = agg_dataframe(df)
    for col in agg_df.columns:
        if col != 'group':
            agg_df[col] = agg_df[col].apply(lambda x: round(x, 2))
    cols = [col for col in agg_df.columns if 'arpd' in col or col == 'group']
    agg_df[cols].to_csv(filename, index=False)

In [9]:
export_arpd(dpm_merged, 'dpm-arpd-summary.csv')
export_arpd(dfm_merged, 'dfm-arpd-summary.csv')

In [10]:
export_arpd(custom_merged, 'custom-arpd-summary.csv')

In [9]:
agg_dataframe(dfm_merged)

,group,rel_gap_neh,arpd_neh,rel_gap_neh_ls,arpd_neh_ls,rel_gap_multistart,arpd_multistart,rel_gap_iga,arpd_iga
0,1,4.857,4.857,2.255,2.255,0.351,0.351,0.151,0.151
1,2,5.307,5.307,3.755,3.755,0.444,0.444,0.168,0.168
2,3,5.248,4.429,3.383,2.571,1.174,0.384,0.985,0.194
3,4,3.163,3.106,2.068,2.012,0.242,0.186,0.205,0.149
4,5,5.334,4.697,3.404,2.782,1.801,1.185,1.259,0.649
5,6,11.756,5.332,10.237,3.899,7.341,1.171,6.092,0.000


In [10]:
agg_dataframe(dpm_merged)

,group,rel_gap_neh,arpd_neh,rel_gap_neh_ls,arpd_neh_ls,rel_gap_multistart,arpd_multistart,rel_gap_iga,arpd_iga
0,1,3.974,3.974,2.445,2.445,0.614,0.614,0.156,0.156
1,2,4.857,4.857,2.872,2.872,0.402,0.402,0.318,0.318
2,3,5.613,4.733,3.530,2.679,1.260,0.432,1.183,0.355
3,4,2.671,2.671,1.686,1.686,0.598,0.598,0.208,0.208
4,5,5.335,5.018,3.647,3.332,1.699,1.388,1.094,0.787
5,6,12.024,5.848,10.033,3.973,7.072,1.180,5.823,0.003
